In [1]:
import pandas as pd
from tokenizers.processors import TemplateProcessing
from tokenizers import Tokenizer
from tokenizers.normalizers import (Sequence,Lowercase, NFD, StripAccents)
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.decoders import BPEDecoder
from tokenizers.trainers import BpeTrainer

In [2]:
data = pd.read_csv('dataset.csv')
data.head()

,text,label,link
0,В 2003-ем году под руководством малоизвестного...,0,dataset/neg/1000083-0.txt
1,"Грустно и печально. Грустно от того, что довол...",0,dataset/neg/1000083-1.txt
2,Давным-давно Кира Найтли ворвалась на экран от...,0,dataset/neg/1000125-3.txt
3,"Я, в общем, ничего против уравновешенного феми...",0,dataset/neg/1000125-4.txt
4,"Измена один из сюжетов, который всегда будет ...",0,dataset/neg/1000125-6.txt


# BPE

In [3]:
special_tokens=["[UNK]","[CLS]","[SEP]","[PAD]","[MASK]"]

temp_proc= TemplateProcessing(
                single = "[CLS] $A [SEP]",
                pair = "[CLS] $A [SEP] $B:1 [SEP]:1",
                special_tokens = [
                            ("[CLS]", special_tokens.index("[CLS]")),
                            ("[SEP]", special_tokens.index("[SEP]")),
                                 ],
                )

tokenizer = Tokenizer(BPE())
tokenizer.normalizer = Sequence(
                                [NFD(), # нормализатор Unicode
                                Lowercase(), # нижний регистр
                                StripAccents()] # удаление доп. буквенных символов (напрмер волну над буквами в турецком языке)
                            )
tokenizer.pre_tokenizer = Whitespace() # разбиение на пробелы
tokenizer.decoder = BPEDecoder() # совместимый с BPE декодер
tokenizer.post_processor = temp_proc


trainer = BpeTrainer(
    vocab_size = 30000,
    special_tokens = special_tokens
)

tokenizer.train_from_iterator(data['text'].to_list(), trainer = trainer)
print(f"Trained vocab size:{tokenizer.get_vocab_size()}" )

Trained vocab size:30000


## Проверка работоспособности токенизатора

Простое предложение

In [4]:
text = 'После этого момента, я больше не хочу есть киви'
text_encode = tokenizer.encode(text)
print(text_encode.tokens)

['[CLS]', 'после', 'этого', 'момента', ',', 'я', 'больше', 'не', 'хочу', 'есть', 'ки', 'ви', '[SEP]']


сложные слова, которые токенизатор не знает

In [5]:
text = 'Ненавижу трансформеры, а особенно их скачивать с Hugging Face'
text_encode = tokenizer.encode(text)
print(text_encode.tokens)

['[CLS]', 'ненавижу', 'трансформеры', ',', 'а', 'особенно', 'их', 'ска', 'чивать', 'с', 'h', 'u', 'g', 'g', 'ing', 'f', 'ace', '[SEP]']


несколько предложений

In [6]:
text_encode = tokenizer.encode('Ненавижу трансформеры',
                               'Но они полезные!')
print(text_encode.tokens)

['[CLS]', 'ненавижу', 'трансформеры', '[SEP]', 'но', 'они', 'полез', 'ные', '!', '[SEP]']


Сохранение токенизатора (словарь и правила слияния)

In [7]:
tokenizer.model.save('tokenizer_BPE')

['tokenizer_BPE\\vocab.json', 'tokenizer_BPE\\merges.txt']

Сохранение всего токенизатора

In [8]:
tokenizer.save('Kinopoisk_tokenizer_BPE.json')

Использование токенизатора после сохранения

In [9]:
tokenizerFromFile = Tokenizer.from_file("Kinopoisk_tokenizer_BPE.json")
text_encode = tokenizerFromFile.encode("Ненавижу трансформеры, а особенно их скачивать с Hugging Face")
print(text_encode.tokens)

['[CLS]', 'ненавижу', 'трансформеры', ',', 'а', 'особенно', 'их', 'ска', 'чивать', 'с', 'h', 'u', 'g', 'g', 'ing', 'f', 'ace', '[SEP]']


# WordPiece

In [11]:
from tokenizers.models import WordPiece
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers.normalizers import BertNormalizer
from tokenizers.trainers import WordPieceTrainer


tokenizer = Tokenizer(WordPiece())
tokenizer.normalizer = BertNormalizer() # очистка
tokenizer.pre_tokenizer = Whitespace()
tokenizer.decoder = WordPieceDecoder()


trainer = WordPieceTrainer(
    vocab_size = 30000,
    special_tokens = ["[UNK]","[CLS]","[SEP]","[PAD]","[MASK]"])

tokenizer.train_from_iterator(data['text'].to_list(), trainer = trainer)
output = tokenizer.encode("Ненавижу трансформеры, а особенно их скачивать с Hugging Face")
print(output.tokens)

['ненавижу', 'трансформеры', ',', 'а', 'особенно', 'их', 'ска', '##чивать', 'с', 'h', '##u', '##g', '##g', '##ing', 'f', '##ace']


проверка UNK токенов

In [14]:
tokenizer.encode("Kralsın aslansın Macbeth!").tokens

['[UNK]', '[UNK]', 'm', '##ac', '##b', '##et', '##h', '!']